In [1]:
# read in data, label with isolation forest, and filter the anomalies.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans

mon = pd.read_csv('~/Desktop/data/netFlow-02-13-2017.csv', sep = ',')
mon_collab = mon[(((mon.srcAddr == '128.143.2.75') & (mon.srcPort == 443)) | ((mon.dstAddr =='128.143.2.75') & (mon.dstPort == 443)))]
mon_collab_clean = mon_collab[["bytes","pkts","dur","rate"]]

In [ ]:
# instead of port 443 we look at port 80
mon_collab = mon[(((mon.srcAddr == '128.143.2.75') & (mon.srcPort == 80)) | ((mon.dstAddr =='128.143.2.75') & (mon.dstPort == 80)))]
mon_collab_clean = mon_collab[["bytes","pkts","dur","rate"]]

In [3]:
len(mon_collab_clean)

474779

In [4]:
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(mon_collab_clean)
y_pred = clf.predict(mon_collab_clean)
mon_collab_clean['ISO_label'] = y_pred
anomalies = mon_collab_clean[mon_collab_clean.ISO_label == -1]
len(anomalies)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
mon_collab_clean.head()

,bytes,pkts,dur,rate,ISO_label
14,3000,28,0.089,269662.96,1
20,31672,52,0.181,1399867.45,1
28,8684,44,0.942,73749.48,1
43,2664,28,0.073,291945.39,1
92,10172,44,0.215,378493.17,1


In [6]:
anomalies = mon_collab_clean[mon_collab_clean.ISO_label == -1]

In [7]:
len(anomalies)

23739

In [10]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial import distance
## function adopted from paper, KSE
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [None] * len(df) # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        kse_score[i] = average
        if (i%100 == 0):
            print('loop %s' % i)
    return kse_score 

In [11]:
kse_scores = KSE_Test(anomalies, 200)

half way there
loop 0
loop 100
loop 200
loop 300
loop 400
loop 500
loop 600
loop 700
loop 800
loop 900
loop 1000
loop 1100
loop 1200
loop 1300
loop 1400
loop 1500
loop 1600
loop 1700
loop 1800
loop 1900
loop 2000
loop 2100
loop 2200
loop 2300
loop 2400
loop 2500
loop 2600
loop 2700
loop 2800
loop 2900
loop 3000
loop 3100
loop 3200
loop 3300
loop 3400
loop 3500
loop 3600
loop 3700
loop 3800
loop 3900
loop 4000
loop 4100
loop 4200
loop 4300
loop 4400
loop 4500
loop 4600
loop 4700
loop 4800
loop 4900
loop 5000
loop 5100
loop 5200
loop 5300
loop 5400
loop 5500
loop 5600
loop 5700
loop 5800
loop 5900
loop 6000
loop 6100
loop 6200
loop 6300
loop 6400
loop 6500
loop 6600
loop 6700
loop 6800
loop 6900
loop 7000
loop 7100
loop 7200
loop 7300
loop 7400
loop 7500
loop 7600
loop 7700
loop 7800
loop 7900
loop 8000
loop 8100
loop 8200
loop 8300
loop 8400
loop 8500
loop 8600
loop 8700
loop 8800
loop 8900
loop 9000
loop 9100
loop 9200
loop 9300
loop 9400
loop 9500
loop 9600
loop 9700
loop 9800
loop 99

In [19]:
anom = []
for i in kse_scores:
    if i > 0.5:
        anom.append(i)

In [28]:
len(anom) * 100 / 474779

0.0004212486230435634

[(1, 2), (2, 3)]